In [ ]:
! pip install Flask


In [ ]:
! pip install import_ipynb

In [ ]:
! pip install pymysql

In [ ]:
! pip install WTForms

In [ ]:
! pip install mysqlclient

In [ ]:
! pip install flask-mysqldb


In [ ]:
! pip install flask-login

In [ ]:
! pip freeze

In [41]:
from flask import Flask, request, render_template, redirect, url_for, jsonify
import import_ipynb
import forms, hashlib
from flask_mysqldb import MySQL

In [42]:
# intilize flask application
app = Flask(__name__)

In [43]:
app.config["SECRET_KEY"]="Hola como etas muy bein y tu?"
app.config['MYSQL_HOST']="localhost"
app.config["MYSQL_USER"]="root"
app.config["MYSQL_PASSWORD"]=""
app.config["MYSQL_DB"]="schoolapp"

In [44]:
mysql = MySQL(app)

In [45]:
@app.route("/")
def home():
    return redirect(url_for("login"))

In [46]:
@app.route("/login",methods=["GET","POST"])
def login():
    if request.method=="POST":
        login_form=forms.LoginForm(request.form)
        if login_form.validate():
            return jsonify(login_form.data)
        else :
            return jsonify({"errors": login_form.errors}),400
    else :
        return render_template("auth/user.html")

In [47]:

@app.route("/register",methods=["GET","POST"])
def register():
    if request.method=="POST":
        print(request.form)
        register_form=forms.RegistrationForm(request.form)
        print(register_form.data)
        if register_form.validate():
            # return jsonify(register_form.data)
            user_data= request.form
            first_name=user_data["first_name"]
            last_name=user_data["last_name"]
            middle_name=user_data["middle_name"]
            email=user_data["email"]
            password=user_data["password"]
            phone_number=user_data["phone_number"]
            hash=password+app.secret_key
            hash=hashlib.sha1(hash.encode())
            password=hash.digest()
            cursor=mysql.connection.cursor()
            cursor.excute("select email from users where email=%s",(email,))
            account = cursor.fetchone()
            if account:
                return "account exist"
            else: 
                return "account doesnt exist"


        else :
            return jsonify({"errors" : register_form.errors }), 400
    else :
        return render_template("auth/registartion.html")



In [48]:
if __name__== "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [20/Dec/2024 21:00:17] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2024 21:00:17] "GET /static/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [20/Dec/2024 21:00:17] "GET /static/bootstrap.js HTTP/1.1" 304 -
